# 0. Intro

Notebook to deploy and evaluate the optimized pipelines

# 1. Packages & Basic Settings

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import shap
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
from evaluation import CustomEval

In [3]:
data_path = '../data'

# 2. Data

& Pipelines instructions

In [4]:
preprocessed_df_filename = 'df_preprocessed.parquet'

df = pd.read_parquet(os.path.join(data_path, 'intermediate', preprocessed_df_filename))

In [5]:
pipelines_instructions = pickle.load(open(os.path.join(data_path, 'output', 'NLP_FSA_pipelines_instructions.pkl'), 'rb'))

# 3. Data Segmentation

In [6]:
overall_test_size = 0.2

split_point = int(round(len(df)*(1-overall_test_size)))

In [7]:
df_train = df.iloc[:split_point].copy()

df_test = df.iloc[split_point:].copy()


# 4. Deploy & Evaluate 

In [8]:
test_rets = df_test['return'].values

selected_metrics = ['MCC','Accuracy','F1','Roc_Auc','PnL_sum','MDD']



In [9]:
extended_names = {'cv': 'CountVectorizer',
                  'tfidf': 'TfidfVectorizer',
                  'binary':'Binary',
                  'ternary':'Ternary',
                  'DT': 'DecisionTree',
                  'RF': 'RandomForest',
                  'HGB': 'GradientBoosting',
                  'VC': 'VotingClassifier'}

In [11]:
all_results = []

for clf, pipes in pipelines_instructions.items():

    for (vect,model), pipe_steps in pipes.items():

        pipe_points = {'Classification':extended_names[clf], 'Vectorizer':extended_names[vect], 'Model':extended_names[model]}

        pipe = Pipeline(pipe_steps)

        pipe.fit(df_train['clean_text'], df_train[f'{clf}_label'])

        y_pred = pipe.predict(df_test['clean_text'])

        computed_metrics = CustomEval(clf=clf, metrics = selected_metrics).eval(df_test[f'{clf}_label'].values, y_pred, test_rets)

        results_store = dict(**pipe_points, **computed_metrics) 

        all_results.append(results_store)


In [12]:
df_res = pd.DataFrame(all_results).sort_values(by=['Classification','Vectorizer']).round(4)

In [13]:
df_res

,Classification,Vectorizer,Model,MCC,F1,Roc_Auc,Accuracy,MDD,PnL_sum
0,Binary,CountVectorizer,DecisionTree,0.7539,0.8973,0.8554,0.8724,-0.0271,3.1677
1,Binary,CountVectorizer,RandomForest,0.7186,0.8758,0.8603,0.8612,-0.0271,3.1352
2,Binary,CountVectorizer,GradientBoosting,0.7935,0.9112,0.8955,0.8987,-0.0271,3.1978
6,Binary,CountVectorizer,VotingClassifier,0.7486,0.8949,0.8676,0.8762,-0.0271,3.1648
3,Binary,TfidfVectorizer,DecisionTree,0.7129,0.8788,0.8527,0.8593,-0.0229,3.1674
4,Binary,TfidfVectorizer,RandomForest,0.7119,0.8716,0.8575,0.8574,-0.0150,3.1702
5,Binary,TfidfVectorizer,GradientBoosting,0.7635,0.9006,0.8763,0.8837,-0.0098,3.2887
7,Binary,TfidfVectorizer,VotingClassifier,0.7052,0.8756,0.8489,0.8555,-0.0229,3.1857
8,Ternary,CountVectorizer,DecisionTree,0.7315,NaN,NaN,0.8255,-0.0271,3.1262
9,Ternary,CountVectorizer,RandomForest,0.7085,NaN,NaN,0.8105,-0.0271,3.0648
